In [9]:
import os

USER = os.getenv("MERCADONA_USER", "ALBERTRQ11@GMAIL.COM")
PASS = os.getenv("MERCADONA_PASS", "15302ARq")

In [10]:
# Requisitos: pip install undetected-chromedriver selenium
import os, time, pathlib, undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- CONFIG ---
URL_SEARCH = "https://tienda.mercadona.es/search-results?query=atun"  # fuerza modal CP
URL_CLASICA_LOGIN = "https://www.mercadona.es/ns/entrada.php"
CP = "08203"

USUARIO  = "albertrq11@gmail.com"   # ⚠️ solo local
PASSWORD = "15302ARq"               # ⚠️ solo local

def w(d, s=15): 
    return WebDriverWait(d, s)

def shot(d, name):
    p = os.path.join(OUT_DIR, name)
    d.save_screenshot(p)
    print("📸", p)

def human_type(el, text, delay=0.06):
    for ch in text:
        el.send_keys(ch); time.sleep(delay)

def page_is_error(d):
    h = d.page_source.lower()
    return ("codigo de error" in h) or ("código de error" in h)

# --- DRIVER (forzamos versión 139 y sin perfil) ---
opts = uc.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--window-size=1366,900")
# 👇 Desactivar gestor de contraseñas / popups
opts.add_experimental_option("prefs", {
    "credentials_enable_service": False,
    "profile.password_manager_enabled": False
})
driver = uc.Chrome(options=opts, version_main=139, headless=False)


print("1) Abriendo página de búsqueda (dispara modal CP)…")
driver.get(URL_SEARCH)

1) Abriendo página de búsqueda (dispara modal CP)…


In [11]:
# ===== CONTROL DE LA PÁGINA =====
# Cookies (si salen)
try:
    w(driver, 8).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()
    print("✅ Cookies aceptadas")
except Exception:
    print("ℹ️ Sin banner de cookies")

# CP 08203
try:
    ip = w(driver, 12).until(EC.presence_of_element_located((By.XPATH, "//input[@name='postalCode']")))
    ip.clear(); human_type(ip, CP); print("📮 CP:", CP)
    cont = w(driver, 12).until(EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='postal-code-checker-button']")))
    cont.click(); print("➡️ Continuar")
except Exception as e:
    print("⚠️ No se pudo introducir CP automáticamente:", e)

# “Ir a la web clásica” si aparece
try:
    classic = w(driver, 8).until(EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='go-to-classic-button']")))
    classic.click(); print("🔄 Pulsado: Ir a la web clásica")
except Exception:
    print("ℹ️ No apareció botón de web clásica (ok)")

# Asegurar login clásica
if "mercadona.es/ns/" not in driver.current_url:
    driver.get("https://www.mercadona.es/")  # referer
    driver.get(URL_CLASICA_LOGIN)
print("🌐 URL actual:", driver.current_url)

# Login clásica
try:
    u = w(driver, 15).until(EC.presence_of_element_located((By.ID, "username")))
    u.clear(); human_type(u, USUARIO); print("📧 Usuario")

    p = w(driver, 15).until(EC.presence_of_element_located((By.ID, "password")))
    p.clear(); human_type(p, PASSWORD); print("🔑 Password")

    time.sleep(0.7)
    btn = w(driver, 12).until(EC.element_to_be_clickable((By.ID, "ImgEntradaAut")))
    btn.send_keys(Keys.SPACE)  # a veces ignoran click()
    print("🚪 Login enviado")
    time.sleep(3)
except Exception as e:
    print("❌ Fallo rellenando login:", e)

from selenium.webdriver.common.action_chains import ActionChains
ActionChains(driver).send_keys(Keys.ESCAPE).perform()
time.sleep(0.4)

# Reintento si sale la página de error
if page_is_error(driver):
    print("⚠️ Página de error detectada. Reintento único…")
    driver.get(URL_CLASICA_LOGIN)
    time.sleep(1.2)
    try:
        u = w(driver, 15).until(EC.presence_of_element_located((By.ID, "username")))
        u.clear(); human_type(u, USUARIO)
        p = w(driver, 15).until(EC.presence_of_element_located((By.ID, "password")))
        p.clear(); human_type(p, PASSWORD)
        time.sleep(0.7)
        w(driver, 12).until(EC.element_to_be_clickable((By.ID, "ImgEntradaAut"))).send_keys(Keys.SPACE)
        print("✅ Reintento enviado")
        time.sleep(3)
    except Exception as e:
        print("❌ Fallo en reintento:", e)

print("🌐 URL final:", driver.current_url)

ℹ️ Sin banner de cookies
📮 CP: 08203
➡️ Continuar
🔄 Pulsado: Ir a la web clásica
🌐 URL actual: https://www.telecompra.mercadona.es/ns/entrada.php?js=1&nidioma=1
📧 Usuario
🔑 Password
🚪 Login enviado
🌐 URL final: https://www.telecompra.mercadona.es/ns/principal.php


In [12]:
# ===== BUSCAR "atun" y EXTRAER nombre, precio y €/kg (página actual) =====
import re, pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def _num_es(s: str):
    if not s: return None
    s = s.strip().replace("\xa0"," ")
    m = re.search(r"(\d{1,3}(?:\.\d{3})*(?:,\d+)|\d+,\d+|\d+(?:\.\d+)?)", s)
    if not m: return None
    t = m.group(1)
    # miles con punto + coma decimal
    if "," in t and "." in t: t = t.replace(".", "").replace(",", ".")
    elif "," in t:            t = t.replace(",", ".")
    try: return float(t)
    except: return None

def _try_type_here():
    """Intenta escribir 'atun' en el input del contexto actual."""
    try:
        inp = driver.find_element(By.ID, "busc_ref")
    except NoSuchElementException:
        try:
            inp = driver.find_element(By.NAME, "busc_ref")
        except NoSuchElementException:
            return False
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", inp)
    try: inp.clear()
    except: pass
    try: human_type(inp, "leche", 0.04)
    except: inp.send_keys("leche")
    inp.send_keys(Keys.ENTER)
    print("🔎 Búsqueda enviada: leche")
    return True

def _dfs_frames(depth=0, max_depth=5):
    """Recorre todos los frames/iframes hasta encontrar el input."""
    if depth > max_depth:
        return False
    if _try_type_here():
        return True
    frames = driver.find_elements(By.TAG_NAME, "frame") + driver.find_elements(By.TAG_NAME, "iframe")
    for f in frames:
        try:
            driver.switch_to.frame(f)
            if _dfs_frames(depth+1, max_depth):
                return True
        except Exception:
            pass
        finally:
            driver.switch_to.parent_frame()
    return False

def buscar_y_entrar_resultados():
    # ir al frameset si no estamos ya en telecompra
    if "telecompra.mercadona.es/ns/" not in driver.current_url:
        driver.get("https://www.telecompra.mercadona.es/ns/buscador.php")
        time.sleep(0.6)
    driver.switch_to.default_content()
    ok = _dfs_frames()
    if not ok:
        print("⚠️ No encontré el input 'busc_ref' en los frames.")
        return False
    # Cambiar al frame de resultados
    driver.switch_to.default_content()
    try:
        w(driver, 12).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "mainFrame")))
    except TimeoutException:
        w(driver, 12).until(EC.frame_to_be_available_and_switch_to_it((
            By.XPATH, "//frame[contains(@name,'main') or contains(@src,'resultado') or contains(@src,'mostrar') or contains(@src,'productos')]"
        )))
    # Esperar a que aparezca la tabla
    w(driver, 12).until(EC.presence_of_element_located((By.XPATH, "//table[@id='TaulaLlista']//tr[td]")))
    print("✅ Resultados visibles en mainFrame.")
    return True

def extraer_pagina_actual():
    """Extrae nombre, precio y €/kg de la página actual del mainFrame."""
    try:
        driver.switch_to.default_content()
        try:
            w(driver, 2).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "mainFrame")))
        except TimeoutException:
            # por si ya estábamos dentro
            pass

        filas = driver.find_elements(By.XPATH, "//table[@id='TaulaLlista']//tr[td]")
        out = []
        for r in filas:
            # nombre (columna 1)
            name = None
            for xp in [
                ".//td[contains(@headers,'header1')]//span[normalize-space()]",
                ".//td[1]//span[normalize-space()]",
            ]:
                try:
                    name = r.find_element(By.XPATH, xp).text.strip()
                    if name: break
                except: pass

            # precio (span id=txtPrecioX o primer span con €)
            price_text = None
            for xp in [
                ".//span[starts-with(@id,'txtPrecio')]",
                ".//td[contains(@headers,'header2')]//span[contains(@class,'tdcenter')][normalize-space()]",
                ".//span[contains(text(),'€')]",
            ]:
                try:
                    price_text = r.find_element(By.XPATH, xp).text.strip()
                    if price_text: break
                except: pass
            price = _num_es(price_text)

            # €/kg (cubre 'txtprecio_uniXX', 'precio_ud', '€/kg', 'P.N.E.', '1 kg: X Euros')
            ppk_text = None
            for xp in [
                ".//span[starts-with(@id,'txtprecio_uni')]",
                ".//span[contains(@class,'precio_ud')]",
                ".//*[contains(.,'€/kg') or contains(.,'€/kilo')]",
                ".//*[contains(.,'P.N.E.')]",
                ".//*[contains(translate(.,'KG','kg'),'kg') and contains(.,'Euro')]",
            ]:
                try:
                    ppk_text = r.find_element(By.XPATH, xp).text.strip()
                    if ppk_text: break
                except: pass
            price_per_kg = _num_es(ppk_text)

            if name:
                out.append({
                    "name": name,
                    "price": price,
                    "price_text": price_text,
                    "price_per_kg": price_per_kg,
                    "price_per_kg_text": ppk_text
                })

        df_local = pd.DataFrame(out).drop_duplicates().reset_index(drop=True)
        return df_local
    except Exception as e:
        print("⚠️ extraer_pagina_actual() falló:", e)
        # devuelve DF vacío en caso de fallo
        return pd.DataFrame(columns=["name","price","price_text","price_per_kg","price_per_kg_text"])

def _leer_info_pagina():
    driver.switch_to.default_content()
    try:
        w(driver, 6).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "mainFrame")))
    except TimeoutException:
        return (None, None)
    txt = ""
    for xp in ["//span[contains(@class,'numpaginas')]", "//*[contains(text(),'Página') and contains(text(),'de')]"]:
        els = driver.find_elements(By.XPATH, xp)
        if els:
            txt = els[0].text.strip()
            break
    m = re.search(r"Página\s+(\d+)\s+de\s+(\d+)", txt, flags=re.I)
    return (int(m.group(1)), int(m.group(2))) if m else (None, None)

def _ir_siguiente():
    driver.switch_to.default_content()
    try:
        w(driver, 6).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "mainFrame")))
    except TimeoutException:
        return False
    try:
        nxt = w(driver, 3).until(EC.element_to_be_clickable((
            By.XPATH, "//a[@id='NEXT' or contains(@href,'Posterior') or contains(@onclick,'Posterior') or contains(.,'Página siguiente')]"
        )))
    except TimeoutException:
        return False
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", nxt)
    time.sleep(0.15)
    try:
        nxt.click()
    except Exception:
        driver.execute_script("arguments[0].click();", nxt)

    driver.switch_to.default_content()
    try:
        w(driver, 8).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "mainFrame")))
        w(driver, 12).until(EC.presence_of_element_located((By.XPATH, "//table[@id='TaulaLlista']//tr[td]")))
    except TimeoutException:
        pass
    return True

def recolectar_todas_las_paginas(max_pages=50):
    todo = []
    visitadas = 0
    while True:
        visitadas += 1
        pa, pt = _leer_info_pagina()
        info = f"Página {pa} de {pt}" if pa and pt else f"Página {visitadas}"
        df_page = extraer_pagina_actual()
        filas = len(df_page) if isinstance(df_page, pd.DataFrame) else 0
        print(f"📄 {info} | filas: {filas}")
        if isinstance(df_page, pd.DataFrame):
            todo.append(df_page)

        if visitadas >= max_pages:
            print("⛔ Corte de seguridad por max_pages."); break
        if not _ir_siguiente():
            break

    # filtra objetos válidos y no vacíos
    valid = [x for x in todo if isinstance(x, pd.DataFrame)]
    if not valid:
        print("⚠️ No hay páginas válidas para concatenar.")
        return pd.DataFrame(columns=["name","price","price_text","price_per_kg","price_per_kg_text"])

    df_all = pd.concat(valid, ignore_index=True).drop_duplicates().reset_index(drop=True)
    print(f"✅ Total filas tras paginar: {len(df_all)}")
    return df_all

# --- Ejecutar: buscar (ya lo hiciste) y paginar ---
if buscar_y_entrar_resultados():
    df = recolectar_todas_las_paginas()

🔎 Búsqueda enviada: leche
✅ Resultados visibles en mainFrame.
📄 Página 1 de 7 | filas: 20
📄 Página 2 de 7 | filas: 20
📄 Página 3 de 7 | filas: 20
📄 Página 4 de 7 | filas: 20
📄 Página 5 de 7 | filas: 20
📄 Página 6 de 7 | filas: 20
📄 Página 7 de 7 | filas: 18
✅ Total filas tras paginar: 138


In [13]:
df

,name,price,price_text,price_per_kg,price_per_kg_text
0,"AFTERSUN LECHE CALMANTE, ECRAN, BOTELLA 400 ML",6.00,"6,00",100.0,"100 Ml: 1,50 Euros"
1,"ARROZ CON LECHE, HACENDADO, PACK 4 X 130 G - 5...",1.35,"1,35",1.0,"1 Kg: 2,597 Euros"
2,BARRITA CEREALES ARROZ Y TRIGO INTEGRAL CHOCOL...,1.55,"1,55",1.0,"1 Kg: 12,917 Euros"
3,BARRITAS CRUJIENTES DE BARQUILLO COBERTURA CHO...,2.80,"2,80",1.0,"1 Kg: 14,00 Euros"
4,BARRITAS CRUJIENTES DE BARQUILLO COBERTURA CHO...,1.80,"1,80",1.0,"1 Kg: 16,072 Euros"
...,...,...,...,...,...
133,"PURE PATATA COPOS SIN LECHE, HACENDADO, 4 SOBR...",2.00,"2,00",1.0,"1 Kg: 4,00 Euros"
134,"QUESO CURADO VACA LECHE CRUDA MORBIER, HACENDA...",4.09,"4,09",1.0,"1 Kg: 20,45 Euros"
135,"SALCHICHA FRANKFURT LECHE, CAMPOFRIO, PAQUETE ...",1.15,"1,15",1.0,"1 Kg: 6,765 Euros"
136,SALSA TIKKA MASALA (CON LECHE DE COCO Y CURRY)...,2.00,"2,00",1.0,"1 Kg: 11,112 Euros"


### Arreglo de la tabla

In [14]:
df.drop(columns=["price_text", "price_per_kg"], inplace=True)
df

,name,price,price_per_kg_text
0,"AFTERSUN LECHE CALMANTE, ECRAN, BOTELLA 400 ML",6.00,"100 Ml: 1,50 Euros"
1,"ARROZ CON LECHE, HACENDADO, PACK 4 X 130 G - 5...",1.35,"1 Kg: 2,597 Euros"
2,BARRITA CEREALES ARROZ Y TRIGO INTEGRAL CHOCOL...,1.55,"1 Kg: 12,917 Euros"
3,BARRITAS CRUJIENTES DE BARQUILLO COBERTURA CHO...,2.80,"1 Kg: 14,00 Euros"
4,BARRITAS CRUJIENTES DE BARQUILLO COBERTURA CHO...,1.80,"1 Kg: 16,072 Euros"
...,...,...,...
133,"PURE PATATA COPOS SIN LECHE, HACENDADO, 4 SOBR...",2.00,"1 Kg: 4,00 Euros"
134,"QUESO CURADO VACA LECHE CRUDA MORBIER, HACENDA...",4.09,"1 Kg: 20,45 Euros"
135,"SALCHICHA FRANKFURT LECHE, CAMPOFRIO, PAQUETE ...",1.15,"1 Kg: 6,765 Euros"
136,SALSA TIKKA MASALA (CON LECHE DE COCO Y CURRY)...,2.00,"1 Kg: 11,112 Euros"
